In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras as tf_keras

In [2]:
(X_train, y_train), (X_test, y_test) = tf_keras.datasets.mnist.load_data()

In [3]:
# 데이터 확인

print( X_train.shape )
X_train[0, :10, :10]

(60000, 28, 28)


array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,  30,  36],
       [  0,   0,   0,   0,   0,   0,   0,  49, 238, 253],
       [  0,   0,   0,   0,   0,   0,   0,  18, 219, 253],
       [  0,   0,   0,   0,   0,   0,   0,   0,  80, 156]], dtype=uint8)

In [4]:
# 데이터 변환 0 ~ 255 -> 0 ~ 1
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

# X_train_scaled[0, 10:20, 10:20]
X_train_scaled.min(), X_train_scaled.max()

(0.0, 1.0)

In [5]:
# 모델 구조 설계

model = tf_keras.models.Sequential()
model.add(tf_keras.layers.Input(shape=(28, 28, 1)))
model.add(tf_keras.layers.Conv2D(filters=32,
                                 kernel_size=3, # 3 x 3 크기의 필터 사용
                                 padding='same', # zero padding을 적용해서 입력 이미지의 크기와 출력 피처맵의 크기를 같게 설정
                                 activation='relu'))
model.add(tf_keras.layers.MaxPool2D(pool_size=2))
model.add(tf_keras.layers.Conv2D(filters=64,
                                 kernel_size=3,
                                 padding='same',
                                 activation='relu'))
model.add(tf_keras.layers.MaxPool2D(pool_size=2))

model.add(tf_keras.layers.Flatten())

model.add(tf_keras.layers.Dense(units=128, activation='relu'))
model.add(tf_keras.layers.Dense(units=10, activation='softmax'))

model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 128)              

In [6]:
# 모델 학습 설계

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
# 모델 학습 (훈련)

fit_history = model.fit(X_train_scaled, y_train, batch_size=64, epochs=20, validation_split=0.2)

Epoch 1/20


750/750 [==============================] - 16s 20ms/step - loss: 0.1786 - accuracy: 0.9449 - val_loss: 0.0609 - val_accuracy: 0.9831
Epoch 2/20
750/750 [==============================] - 15s 19ms/step - loss: 0.0515 - accuracy: 0.9844 - val_loss: 0.0518 - val_accuracy: 0.9839
Epoch 3/20
750/750 [==============================] - 15s 20ms/step - loss: 0.0340 - accuracy: 0.9895 - val_loss: 0.0445 - val_accuracy: 0.9863
Epoch 4/20
750/750 [==============================] - 15s 20ms/step - loss: 0.0279 - accuracy: 0.9909 - val_loss: 0.0380 - val_accuracy: 0.9885
Epoch 5/20
750/750 [==============================] - 15s 20ms/step - loss: 0.0196 - accuracy: 0.9938 - val_loss: 0.0363 - val_accuracy: 0.9902
Epoch 6/20
750/750 [==============================] - 15s 20ms/step - loss: 0.0142 - accuracy: 0.9952 - val_loss: 0.0483 - val_accuracy: 0.9873
Epoch 7/20
750/750 [==============================] - 15s 19ms/step - loss: 0.0119 - accuracy: 0.9959 - val_loss: 0.0377 - val_accurac

In [8]:
model.evaluate(X_test_scaled, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.0462 - accuracy: 0.9906


[0.04622270166873932, 0.9905999898910522]

In [9]:
# 모델 저장
import pickle

model.save("models/mnist-cnn-model.keras")

with open("models/mnist-cnn-model2.keras", "wb") as f:
    pickle.dump(model, f)

In [15]:
# 저장된 모델 가져오기

model2 = tf_keras.models.load_model("models/mnist-cnn-model.keras")

with open("models/mnist-cnn-model2.keras", "rb") as f:
    model3 = pickle.load(f)

In [16]:
model2.evaluate(X_test_scaled, y_test), model3.evaluate(X_test_scaled, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9892 - loss: 0.0509
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9892 - loss: 0.0509


([0.040924008935689926, 0.9919000267982483],
 [0.040924008935689926, 0.9919000267982483])

In [12]:
np.argmax(model.predict(X_test_scaled[:1])), np.max(model.predict(X_test_scaled[:1]))

1/1 [==============================] - 0s 30ms/step


(7, 1.0)

In [15]:
X_train_scaled[0].shape

(28, 28)

In [26]:
from PIL import Image

img = Image.fromarray(X_train_scaled[0] * 255)
img = img.convert("L")
img.save('digit.png')

In [25]:
np.max(img.getdata() * 255)

TypeError: unsupported operand type(s) for *: 'ImagingCore' and 'int'